# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [63]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-559ada6c32-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


# History Workflow Series ("Cultural Movements explorative search") 

Consider the following exploratory information need:

> investigate the cultural movements, in particular the Renaissance, Baroque and Romanticsm. We'd like to get an overview of the main artists belonging to these movements, musicians, poets, sculptors and painters.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q37562`| Donatello    | node |
| `wd:Q19675`| Louvre Museum | node |





Also consider

```
wd:Q37562 ?p ?obj .
```

is the BGP to retrieve all **properties of Donatello**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for Cultural Movement

2. Identify the BGP for Renassaince, Baroque and Romanticism

3. Identify the BGP for composer, painter, sculptor and poet

4. Consider Renaissance, Baroque and Romanticism. Find how many composers, painters, sculptors and poets belong to those cultural movements (the results set must be a list of triples movement, job, #people)

5. Find how many sculptures were made in Italy during Renaissance, Baroque, and Romanticism movements. Please consider all the sculptures and all the sculpture subtypes. (the result set must be a list of triples movement IRI, label and #sculptures).

6. For each European country find how many poets belong to Romanticism (the result set must be a list of triples country IRI, label and #poets).

7. Find the number of sculptures in the Louvre Museum belonging to Renaissance, Baroque, or Romanticism (return a number for each movement) (the result set must be a list of triples movement IRI, label and #sculptures).

## Task 1
Identify the BGP for Cultural Movement

In [64]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q37562 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1066'), ('name', 'student of')]
[('p', 'http://www.wikidata.org/prop/direct/P1138'), ('name', 'Kunstindeks Danmark Artist ID')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('name', 'place of burial')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1248'), ('name', 'KulturNav-ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), (

In [10]:
# query example
# [('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q37562 wdt:P135 ?p .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
1


In [11]:
# query example
# [('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
#[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1472236 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P5604'), ('name', 'archINFORM keyword ID')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('name', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P576'), ('name', 'dissolved, abolished or demolished date')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', '

In [43]:
# query example
# [('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
#[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
#[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?p wdt:P910 ?o  .
   # get the label
   ?p sc:name ?name.
   FILTER(REGEX(?name,"cultural movement"^^xsd:string))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q2198855'), ('name', 'cultural movement')]
1


In [60]:
# write your queries
# [('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
#[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q4692 ?obj ?p .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q13830551'), ('name', 'Portal:Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q15885248'), ('name', 'Renaissance painting')]
[('p', 'http://www.wikidata.org/entity/Q1781608'), ('name', 'Renaissance literature')]
[('p', 'http://www.wikidata.org/entity/Q1404472'), ('name', 'Italian Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q1474884'), ('name', 'High Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q2041543'), ('name', 'Ottův slovník naučný')]
[('p', 'http://www.wikidata.org/entity/Q27496'), ('name', 'Western Europe')]
[('p', 'http://www.wikidata.org/entity/Q3181656'), ('name', 'The Nuttall Encyclopædia')]
[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
[('p', 'http://www.wikidata.org/entity/Q5460604'), ('name', 'Wikipedia:Vital articles')]
[('p', 'http://www.wikidata.org/entity/Q602358'), ('name', 'Brockhaus and Efro

Final query for this task

In [8]:
# write your final query
# [('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
#[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
#[('p', 'http://www.wikidata.org/entity/Q2198855'), ('name', 'cultural movement')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q2198855 ?obj ?p .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/P135'), ('name', 'movement')]
[('p', 'http://www.wikidata.org/entity/Q49773'), ('name', 'social movement')]
[('p', 'http://www.wikidata.org/entity/Q3533467'), ('name', 'group action')]
[('p', 'http://www.wikidata.org/entity/Q6638204'), ('name', 'Category:Cultural movement')]
4


## Task 2
Identify the BGP for Renassaince, Baroque and Romanticism

In [ ]:
# write your queries
# [('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
#[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1472236 wdt:P361 ?p .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
1


In [ ]:
# write your queries
# [('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
#[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q4692 ?obj ?p .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q13830551'), ('name', 'Portal:Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q15885248'), ('name', 'Renaissance painting')]
[('p', 'http://www.wikidata.org/entity/Q1781608'), ('name', 'Renaissance literature')]
[('p', 'http://www.wikidata.org/entity/Q1404472'), ('name', 'Italian Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q1474884'), ('name', 'High Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q2041543'), ('name', 'Ottův slovník naučný')]
[('p', 'http://www.wikidata.org/entity/Q27496'), ('name', 'Western Europe')]
[('p', 'http://www.wikidata.org/entity/Q3181656'), ('name', 'The Nuttall Encyclopædia')]
[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
[('p', 'http://www.wikidata.org/entity/Q5460604'), ('name', 'Wikipedia:Vital articles')]
[('p', 'http://www.wikidata.org/entity/Q602358'), ('name', 'Brockhaus and Efro

In [ ]:
# write your queries
# [('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
#[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
#[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
queryString = """
SELECT DISTINCT  ?o ?bname
WHERE {
   # bind something
  # wd:Q4692 ?obj ?p .
   wd:Q37853 ?obj1 ?o.
   # get the label
   #?p sc:name ?name.
   ?o sc:name ?bname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q172991'), ('bname', 'Council of Trent')]
[('o', 'http://www.wikidata.org/entity/Q11514315'), ('bname', 'historical period')]
[('o', 'http://www.wikidata.org/entity/Q1292119'), ('bname', 'style')]
[('o', 'http://www.wikidata.org/entity/Q131808'), ('bname', 'Mannerism')]
[('o', 'http://www.wikidata.org/entity/Q2041543'), ('bname', 'Ottův slovník naučný')]
[('o', 'http://www.wikidata.org/entity/Q2657718'), ('bname', 'Armenian Soviet Encyclopedia')]
[('o', 'http://www.wikidata.org/entity/Q3181656'), ('bname', 'The Nuttall Encyclopædia')]
[('o', 'http://www.wikidata.org/entity/Q32880'), ('bname', 'architectural style')]
[('o', 'http://www.wikidata.org/entity/Q4692'), ('bname', 'Renaissance')]
[('o', 'http://www.wikidata.org/entity/Q968159'), ('bname', 'art movement')]
[('o', 'http://www.wikidata.org/entity/Q2198855'), ('bname', 'cultural movement')]
[('o', 'http://www.wikidata.org/entity/Q1431376'), ('bname', 'Baroque literature')]
[('o', 'htt

In [31]:
# write your final query
# [('p', 'http://www.wikidata.org/prop/direct/P135'), ('name', 'movement')]
#[('p', 'http://www.wikidata.org/entity/Q1472236'), ('name', 'Early Renaissance')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
#[('p', 'http://www.wikidata.org/entity/Q2198855'), ('name', 'cultural movement')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
    ?p wdt:P31 wd:Q2198855.
    #wd:Q2198855 ?obj ?p.
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q8210150'), ('name', 'Avanzada Regia')]
[('p', 'http://www.wikidata.org/entity/Q211481'), ('name', 'Dogme 95')]
[('p', 'http://www.wikidata.org/entity/Q2046861'), ('name', 'Palaeologan Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q1400430'), ('name', 'Macedonian Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q430167'), ('name', 'Spanish Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q1814286'), ('name', 'Young Estonia')]
[('p', 'http://www.wikidata.org/entity/Q2614483'), ('name', 'Pre-romanticism')]
[('p', 'http://www.wikidata.org/entity/Q39427'), ('name', 'surrealism')]
[('p', 'http://www.wikidata.org/entity/Q2349908'), ('name', 'French Renaissance')]
[('p', 'http://www.wikidata.org/entity/Q878985'), ('name', 'modernism')]
[('p', 'http://www.wikidata.org/entity/Q466887'), ('name', 'Belle Époque')]
[('p', 'http://www.wikidata.org/entity/Q12539'), ('name', 'Age of Enlightenment')]
[('p', 'http://www.wikidata.org/entit

In [33]:
#[('p', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
#[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
queryString = """
SELECT DISTINCT ?roman ?rname ?bar ?bname ?rena ?ename 
WHERE {
   # bind something
    wd:Q37068 ?obj ?roman.
    wd:Q37853 ?obj1 ?bar.
    wd:Q4692 ?obj2 ?rena.
   # get the label
   ?roman sc:name ?rname.
   ?bar sc:name ?bname.
   ?rena sc:name ?ename.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('roman', 'http://www.wikidata.org/entity/Q2614483'), ('rname', 'Pre-romanticism'), ('bar', 'http://www.wikidata.org/entity/Q2041543'), ('bname', 'Ottův slovník naučný'), ('rena', 'http://www.wikidata.org/entity/Q12554'), ('ename', 'Middle Ages')]
[('roman', 'http://www.wikidata.org/entity/Q2614483'), ('rname', 'Pre-romanticism'), ('bar', 'http://www.wikidata.org/entity/Q2041543'), ('bname', 'Ottův slovník naučný'), ('rena', 'http://www.wikidata.org/entity/Q27496'), ('ename', 'Western Europe')]
[('roman', 'http://www.wikidata.org/entity/Q2614483'), ('rname', 'Pre-romanticism'), ('bar', 'http://www.wikidata.org/entity/Q2041543'), ('bname', 'Ottův slovník naučný'), ('rena', 'http://www.wikidata.org/entity/Q979160'), ('ename', 'Proto-Renaissance')]
[('roman', 'http://www.wikidata.org/entity/Q2614483'), ('rname', 'Pre-romanticism'), ('bar', 'http://www.wikidata.org/entity/Q2041543'), ('bname', 'Ottův slovník naučný'), ('rena', 'http://www.wikidata.org/entity/Q1404472'), ('ename', 

Final query for this task

In [66]:
# write your final query
#[('p', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
#[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
queryString = """
SELECT DISTINCT ?roman ?rname ?bar ?bname ?rena ?ename 
WHERE {
    {
    wd:Q37068 ?roman ?obj.
    ?roman sc:name ?rname.
    }
    UNION
    {
    wd:Q37853 ?bar ?obj1.
    ?bar sc:name ?bname.
    }
    UNION 
    {
    wd:Q4692 ?rena ?obj2.
    ?rena sc:name ?ename.
   }
   
   
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('roman', 'http://www.wikidata.org/prop/direct/P1014'), ('rname', 'Art & Architecture Thesaurus ID')]
[('roman', 'http://www.wikidata.org/prop/direct/P1036'), ('rname', 'Dewey Decimal Classification')]
[('roman', 'http://www.wikidata.org/prop/direct/P1051'), ('rname', 'PSH ID')]
[('roman', 'http://www.wikidata.org/prop/direct/P1245'), ('rname', 'OmegaWiki Defined Meaning')]
[('roman', 'http://www.wikidata.org/prop/direct/P1296'), ('rname', 'Gran Enciclopèdia Catalana ID')]
[('roman', 'http://www.wikidata.org/prop/direct/P1343'), ('rname', 'described by source')]
[('roman', 'http://www.wikidata.org/prop/direct/P1417'), ('rname', 'Encyclopædia Britannica Online ID')]
[('roman', 'http://www.wikidata.org/prop/direct/P155'), ('rname', 'follows')]
[('roman', 'http://www.wikidata.org/prop/direct/P156'), ('rname', 'followed by')]
[('roman', 'http://www.wikidata.org/prop/direct/P18'), ('rname', 'image')]
[('roman', 'http://www.wikidata.org/prop/direct/P1807'), ('rname', 'Great Aragones

## Task 3
Identify the BGP for composer, painter, sculptor and poet

In [54]:
# write your queries
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?obj wdt:P106 ?p .
   # get the label
   ?p sc:name ?name.
   FILTER(REGEX(?name,"composer"^^xsd:string))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q21680663'), ('name', 'classical composer')]
[('p', 'http://www.wikidata.org/entity/Q21680699'), ('name', 'operetta composer')]
[('p', 'http://www.wikidata.org/entity/Q1415090'), ('name', 'film score composer')]
[('p', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
[('p', 'http://www.wikidata.org/entity/Q2627699'), ('name', 'chess composer')]
[('p', 'http://www.wikidata.org/entity/Q21680731'), ('name', 'opera composer')]
[('p', 'http://www.wikidata.org/entity/Q63536580'), ('name', 'video game composer')]
[('p', 'http://www.wikidata.org/entity/Q64356038'), ('name', 'musical theatre composer')]
[('p', 'http://www.wikidata.org/entity/Q108001849'), ('name', 'amateur composer')]
[('p', 'http://www.wikidata.org/entity/Q28771895'), ('name', 'vocal composer')]
[('p', 'http://www.wikidata.org/entity/Q4765165'), ('name', 'Anime composer')]
11


In [55]:
#[('p', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?obj wdt:P106 ?p .
   # get the label
   ?p sc:name ?name.
   FILTER(REGEX(?name,"sculptor"^^xsd:string))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
[('p', 'http://www.wikidata.org/entity/Q10611691'), ('name', 'ornamental sculptor')]
[('p', 'http://www.wikidata.org/entity/Q40749908'), ('name', 'bronze sculptor')]
3


In [56]:
#[('p', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
#[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?obj wdt:P106 ?p .
   # get the label
   ?p sc:name ?name.
   FILTER(REGEX(?name,"painter"^^xsd:string))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q17505911'), ('name', 'architectural painter')]
[('p', 'http://www.wikidata.org/entity/Q11606844'), ('name', 'Buddhist painter')]
[('p', 'http://www.wikidata.org/entity/Q67200599'), ('name', 'Yōga painter')]
[('p', 'http://www.wikidata.org/entity/Q67649782'), ('name', 'finger painter')]
[('p', 'http://www.wikidata.org/entity/Q1743189'), ('name', 'church-painter')]
[('p', 'http://www.wikidata.org/entity/Q59938355'), ('name', 'flower painter')]
[('p', 'http://www.wikidata.org/entity/Q25616553'), ('name', 'marine painter')]
[('p', 'http://www.wikidata.org/entity/Q15301511'), ('name', 'vase painter')]
[('p', 'http://www.wikidata.org/entity/Q15303031'), ('name', 'black-figure vase painter')]
[('p', 'http://www.wikidata.org/entity/Q15303045'), ('name', 'red-figure vase painter')]
[('p', 'http://www.wikidata.org/entity/Q21282419'), ('name', 'Attic vase-painter')]
[('p', 'http://www.wikidata.org/entity/Q41000669'), ('name', 'white-groud vase-paint

In [57]:
#[('p', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
#[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
#[('p', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   ?obj wdt:P106 ?p .
   # get the label
   ?p sc:name ?name.
   FILTER(REGEX(?name,"poet"^^xsd:string))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q106376813'), ('name', 'visual poet')]
[('p', 'http://www.wikidata.org/entity/Q1209498'), ('name', 'poet lawyer')]
[('p', 'http://www.wikidata.org/entity/Q482'), ('name', 'poetry')]
[('p', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
[('p', 'http://www.wikidata.org/entity/Q182357'), ('name', 'lyric poetry')]
[('p', 'http://www.wikidata.org/entity/Q11386386'), ('name', 'haiku poet')]
[('p', 'http://www.wikidata.org/entity/Q11122954'), ('name', 'tanka poet')]
[('p', 'http://www.wikidata.org/entity/Q23117687'), ('name', 'dithyrambic poet')]
[('p', 'http://www.wikidata.org/entity/Q1237928'), ('name', 'poet laureate')]
[('p', 'http://www.wikidata.org/entity/Q2293636'), ('name', 'slam poet')]
[('p', 'http://www.wikidata.org/entity/Q7168247'), ('name', 'performance poetry')]
[('p', 'http://www.wikidata.org/entity/Q7207537'), ('name', 'poetry reading')]
[('p', 'http://www.wikidata.org/entity/Q107714046'), ('name', 'performance poet'

In [59]:
#[('p', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
#[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
#[('p', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
#[('p', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
queryString = """
SELECT DISTINCT ?com ?cname ?scul ?sname ?paint ?pname ?poet ?poname
WHERE {
   {
    wd:Q36843 ?obj ?com.
    ?com sc:name ?cname.
    }
    UNION
    {
    wd:Q1281618 ?obj1 ?scul.
    ?scul sc:name ?sname.
    }
    UNION 
    {
    wd:Q1028181 ?obj2 ?paint.
    ?paint sc:name ?pname.
   }
   UNION 
    {
    wd:Q49757 ?obj3 ?poet.
    ?poet sc:name ?poname.
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('com', 'http://www.wikidata.org/entity/Q17856046'), ('cname', 'member of the Reichstag of Nazi Germany')]
[('com', 'http://www.wikidata.org/entity/Q708038'), ('cname', 'Kaiser Wilhelm Society')]
[('com', 'http://www.wikidata.org/entity/Q22082717'), ('cname', 'eugenicist')]
[('com', 'http://www.wikidata.org/entity/Q10737'), ('cname', 'suicide')]
[('com', 'http://www.wikidata.org/entity/Q152838'), ('cname', 'University of Göttingen')]
[('com', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany')]
[('com', 'http://www.wikidata.org/entity/Q188'), ('cname', 'German')]
[('com', 'http://www.wikidata.org/entity/Q210392'), ('cname', 'killed in action')]
[('com', 'http://www.wikidata.org/entity/Q2865'), ('cname', 'Kassel')]
[('com', 'http://www.wikidata.org/entity/Q39631'), ('cname', 'physician')]
[('com', 'http://www.wikidata.org/entity/Q41304'), ('cname', 'Weimar Republic')]
[('com', 'http://www.wikidata.org/entity/Q482980'), ('cname', 'author')]
[('com', 'http://www.wikidata

Final query for this task

In [60]:
# write your final query
#[('p', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
#[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
#[('p', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
#[('p', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
queryString = """
SELECT DISTINCT ?com ?cname ?scul ?sname ?paint ?pname ?poet ?poname
WHERE {
   {
    wd:Q36843 ?com ?obj.
    ?com sc:name ?cname.
    }
    UNION
    {
    wd:Q1281618 ?obj1 ?scul.
    ?scul sc:name ?sname.
    }
    UNION 
    {
    wd:Q1028181 ?obj2 ?paint.
    ?paint sc:name ?pname.
   }
   UNION 
    {
    wd:Q49757 ?obj3 ?poet.
    ?poet sc:name ?poname.
   }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('com', 'http://www.wikidata.org/prop/direct/P9044'), ('cname', 'GEPRIS-Historisch ID (Person)')]
[('com', 'http://www.wikidata.org/prop/direct/P1006'), ('cname', 'Nationale Thesaurus voor Auteurs ID')]
[('com', 'http://www.wikidata.org/prop/direct/P1015'), ('cname', 'NORAF ID')]
[('com', 'http://www.wikidata.org/prop/direct/P102'), ('cname', 'member of political party')]
[('com', 'http://www.wikidata.org/prop/direct/P106'), ('cname', 'occupation')]
[('com', 'http://www.wikidata.org/prop/direct/P119'), ('cname', 'place of burial')]
[('com', 'http://www.wikidata.org/prop/direct/P1196'), ('cname', 'manner of death')]
[('com', 'http://www.wikidata.org/prop/direct/P1207'), ('cname', 'NUKAT ID')]
[('com', 'http://www.wikidata.org/prop/direct/P1280'), ('cname', 'CONOR.SI ID')]
[('com', 'http://www.wikidata.org/prop/direct/P1347'), ('cname', 'military casualty classification')]
[('com', 'http://www.wikidata.org/prop/direct/P1412'), ('cname', 'languages spoken, written or signed')]
[(

## Task 4
Consider Renaissance, Baroque and Romanticism. Find how many composers, painters, sculptors and poets belong to those cultural movements (the results set must be a list of triples movement, job, #people)

In [96]:
# Considering Occupation ID for getting all professions 
#
queryString = """
SELECT DISTINCT ?roman ?rname 
WHERE {
    ?r wdt:P106 ?roman.
    ?roman sc:name ?rname.
    FILTER(REGEX(?rname, "poet"^^xsd:string))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('roman', 'http://www.wikidata.org/entity/Q106376813'), ('rname', 'visual poet')]
[('roman', 'http://www.wikidata.org/entity/Q1209498'), ('rname', 'poet lawyer')]
[('roman', 'http://www.wikidata.org/entity/Q482'), ('rname', 'poetry')]
[('roman', 'http://www.wikidata.org/entity/Q49757'), ('rname', 'poet')]
[('roman', 'http://www.wikidata.org/entity/Q182357'), ('rname', 'lyric poetry')]
[('roman', 'http://www.wikidata.org/entity/Q11386386'), ('rname', 'haiku poet')]
[('roman', 'http://www.wikidata.org/entity/Q11122954'), ('rname', 'tanka poet')]
[('roman', 'http://www.wikidata.org/entity/Q23117687'), ('rname', 'dithyrambic poet')]
[('roman', 'http://www.wikidata.org/entity/Q1237928'), ('rname', 'poet laureate')]
[('roman', 'http://www.wikidata.org/entity/Q2293636'), ('rname', 'slam poet')]
[('roman', 'http://www.wikidata.org/entity/Q7168247'), ('rname', 'performance poetry')]
[('roman', 'http://www.wikidata.org/entity/Q7207537'), ('rname', 'poetry reading')]
[('roman', 'http://w

In [195]:
# write your queries
#[('p', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
#[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
#[('NumOf', '156'), ('ocup', 'http://www.wikidata.org/entity/Q1028181'), ('oname', 'painter')]
#[('p', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
#[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
#[('p', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
#[('p', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
queryString = """
SELECT  *
WHERE {
    {SELECT DISTINCT (COUNT(?roman) as ?Num) ?ocup (?oname as ?Romanticism)  WHERE
    {
    ?roman ?obj wd:Q37068;
        wdt:P106 ?ocup.
    ?roman sc:name ?rname.
    ?ocup sc:name ?oname.
    FILTER (REGEX(?oname, "sculptor"^^xsd:string) || REGEX(?oname, "composer"^^xsd:string ) || REGEX(?oname, "poet"^^xsd:string ) || REGEX(?oname, "painter"^^xsd:string ))
    #FILTER (REGEX(?oname, "painter"^^xsd:string ))
    #FILTER (?oname = 'sculptor'^^xsd:string || ?oname = 'painter'^^xsd:string) 
    } GROUP BY ?oname ?ocup
    }
    UNION
    {
    select distinct (COUNT(?bar) as ?Num) ?ocup1 (?oname1 as ?Boroque) where
    {
     ?bar ?obj1 wd:Q37853;
         wdt:P106 ?ocup1.
    #?bar sc:name ?bname.
    ?ocup1 sc:name ?oname1.
    FILTER (REGEX(?oname1, "sculptor"^^xsd:string ) || REGEX(?oname1, "painter"^^xsd:string ) || REGEX(?oname1, "poet"^^xsd:string)  || REGEX(?oname1, "composer"^^xsd:string ))
    } GROUP BY ?oname1 ?ocup1
    }
    UNION
    {
    select distinct (COUNT(?rena) as ?Num) ?ocup2 ?oname2 as ?Renaissance
    {
     ?rena ?obj2 wd:Q4692;
         wdt:P106 ?ocup2.
    ?rena sc:name ?ename.
    ?ocup2 sc:name ?oname2.
    FILTER (REGEX(?oname2, "sculptor"^^xsd:string ) || REGEX(?oname2, "painter"^^xsd:string ) || REGEX(?oname2, "poet"^^xsd:string)  || REGEX(?oname2, "composer"^^xsd:string ))
    #FILTER (?oname2 = "sculptor"^^xsd:string || ?oname2 = "painter"^^xsd:string) 
   } 
   GROUP BY ?oname2 ?ocup2
   }
   
}  
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('Num', '8'), ('ocup', 'http://www.wikidata.org/entity/Q1209498'), ('Romanticism', 'poet lawyer')]
[('Num', '1'), ('ocup', 'http://www.wikidata.org/entity/Q59938438'), ('Romanticism', 'history painter')]
[('Num', '7'), ('ocup', 'http://www.wikidata.org/entity/Q21600439'), ('Romanticism', 'landscape painter')]
[('Num', '21'), ('ocup', 'http://www.wikidata.org/entity/Q36834'), ('Romanticism', 'composer')]
[('Num', '156'), ('ocup', 'http://www.wikidata.org/entity/Q1028181'), ('Romanticism', 'painter')]
[('Num', '1'), ('ocup', 'http://www.wikidata.org/entity/Q21680663'), ('Romanticism', 'classical composer')]
[('Num', '1'), ('ocup', 'http://www.wikidata.org/entity/Q2627699'), ('Romanticism', 'chess composer')]
[('Num', '159'), ('ocup', 'http://www.wikidata.org/entity/Q49757'), ('Romanticism', 'poet')]
[('Num', '14'), ('ocup', 'http://www.wikidata.org/entity/Q1281618'), ('Romanticism', 'sculptor')]
[('Num', '1'), ('ocup1', 'http://www.wikidata.org/entity/Q28947943'), ('Boroque', 'm

Final query for this task

In [237]:
# write your final query
# write your queries
#[('p', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
#[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
#[('NumOf', '156'), ('ocup', 'http://www.wikidata.org/entity/Q1028181'), ('oname', 'painter')]
#[('p', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
#[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
#[('p', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
#[('p', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
queryString = """
SELECT  *
WHERE {
    {SELECT DISTINCT (COUNT(?roman) as ?Num) ?ocup (?oname as ?Romanticism)  WHERE
    {
    ?roman ?obj wd:Q37068;
        wdt:P106 ?ocup.
    ?roman sc:name ?rname.
    ?ocup sc:name ?oname.
    FILTER (REGEX(?oname, "sculptor"^^xsd:string) || REGEX(?oname, "composer"^^xsd:string ) || REGEX(?oname, "poet"^^xsd:string ) || REGEX(?oname, "painter"^^xsd:string ))
    #FILTER (REGEX(?oname, "painter"^^xsd:string ))
    #FILTER (?oname = 'sculptor'^^xsd:string || ?oname = 'painter'^^xsd:string) 
    } GROUP BY ?oname ?ocup
    }
    UNION
    {
    select distinct (COUNT(?bar) as ?Num) ?ocup1 (?oname1 as ?Boroque) where
    {
     ?bar ?obj1 wd:Q37853;
         wdt:P106 ?ocup1.
    #?bar sc:name ?bname.
    ?ocup1 sc:name ?oname1.
    FILTER (REGEX(?oname1, "sculptor"^^xsd:string ) || REGEX(?oname1, "painter"^^xsd:string ) || REGEX(?oname1, "poet"^^xsd:string)  || REGEX(?oname1, "composer"^^xsd:string ))
    } GROUP BY ?oname1 ?ocup1
    }
    UNION
    {
    select distinct (COUNT(?rena) as ?Num) ?ocup2 ?oname2 as ?Renaissance
    {
     ?rena ?obj2 wd:Q4692;
         wdt:P106 ?ocup2.
    ?rena sc:name ?ename.
    ?ocup2 sc:name ?oname2.
    FILTER (REGEX(?oname2, "sculptor"^^xsd:string ) || REGEX(?oname2, "painter"^^xsd:string ) || REGEX(?oname2, "poet"^^xsd:string)  || REGEX(?oname2, "composer"^^xsd:string ))
    #FILTER (?oname2 = "sculptor"^^xsd:string || ?oname2 = "painter"^^xsd:string) 
   } 
   GROUP BY ?oname2 ?ocup2
   }
   
}  
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('Num', '8'), ('ocup', 'http://www.wikidata.org/entity/Q1209498'), ('Romanticism', 'poet lawyer')]
[('Num', '1'), ('ocup', 'http://www.wikidata.org/entity/Q59938438'), ('Romanticism', 'history painter')]
[('Num', '7'), ('ocup', 'http://www.wikidata.org/entity/Q21600439'), ('Romanticism', 'landscape painter')]
[('Num', '21'), ('ocup', 'http://www.wikidata.org/entity/Q36834'), ('Romanticism', 'composer')]
[('Num', '156'), ('ocup', 'http://www.wikidata.org/entity/Q1028181'), ('Romanticism', 'painter')]
[('Num', '1'), ('ocup', 'http://www.wikidata.org/entity/Q21680663'), ('Romanticism', 'classical composer')]
[('Num', '1'), ('ocup', 'http://www.wikidata.org/entity/Q2627699'), ('Romanticism', 'chess composer')]
[('Num', '159'), ('ocup', 'http://www.wikidata.org/entity/Q49757'), ('Romanticism', 'poet')]
[('Num', '14'), ('ocup', 'http://www.wikidata.org/entity/Q1281618'), ('Romanticism', 'sculptor')]
[('Num', '1'), ('ocup1', 'http://www.wikidata.org/entity/Q28947943'), ('Boroque', 'm

## Task 5
Find how many sculptures were made in Italy during Renaissance, Baroque, and Romanticism movements. Please consider all the sculptures and all the sculpture subtypes. (the result set must be a list of triples movement IRI, label and #sculptures).

In [149]:
# write your queries
#[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
queryString = """
SELECT DISTINCT ?scul ?sname
WHERE {
     ?obj1 ?scul wd:Q860861.
    ?scul sc:name ?sname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('scul', 'http://www.wikidata.org/prop/direct/P101'), ('sname', 'field of work')]
[('scul', 'http://www.wikidata.org/prop/direct/P1056'), ('sname', 'product or material produced')]
[('scul', 'http://www.wikidata.org/prop/direct/P106'), ('sname', 'occupation')]
[('scul', 'http://www.wikidata.org/prop/direct/P136'), ('sname', 'genre')]
[('scul', 'http://www.wikidata.org/prop/direct/P138'), ('sname', 'named after')]
[('scul', 'http://www.wikidata.org/prop/direct/P144'), ('sname', 'based on')]
[('scul', 'http://www.wikidata.org/prop/direct/P180'), ('sname', 'depicts')]
[('scul', 'http://www.wikidata.org/prop/direct/P1889'), ('sname', 'different from')]
[('scul', 'http://www.wikidata.org/prop/direct/P195'), ('sname', 'collection')]
[('scul', 'http://www.wikidata.org/prop/direct/P2650'), ('sname', 'interested in')]
[('scul', 'http://www.wikidata.org/prop/direct/P2670'), ('sname', 'has parts of the class')]
[('scul', 'http://www.wikidata.org/prop/direct/P279'), ('sname', 'subclass of

In [264]:
# write your queries
#[('p', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
#[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
#[('NumOf', '156'), ('ocup', 'http://www.wikidata.org/entity/Q1028181'), ('oname', 'painter')]
#[('p', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
#[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
#[('p', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
#[('p', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
# write your queries
#[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
#[('cname', 'http://www.wikidata.org/entity/Q38'), ('m', 'Italy')]
queryString = """
SELECT DISTINCT ?k ?sname
WHERE {
        ?k wdt:P162 wd:Q38.
        ?k sc:name ?sname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('k', 'http://www.wikidata.org/entity/Q2933446'), ('sname', 'Calabuch')]
1


In [260]:
# write your final query
# write your queries
#[('p', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
#[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
#[('NumOf', '156'), ('ocup', 'http://www.wikidata.org/entity/Q1028181'), ('oname', 'painter')]
#[('p', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
#[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
#[('p', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
#[('p', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
queryString = """
SELECT  DISTINCT (COUNT(?roman) as ?Num) ?obj ?rname ?oname ?connection 
WHERE {
    ?roman ?obj wd:Q37068;
        wdt:P106 wd:Q1281618.
        
      #wd:Q1281618 ?coun wd:Q38.
        
    ?roman sc:name ?rname.
    ?obj sc:name ?oname.
   # ?coun sc:name ?connection
     
   }
 
GROUP BY ?obj ?rname ?oname ?connection
LIMIT 500
"""

print("Results")
x=run_query(queryString)

Results
[('Num', '1'), ('obj', 'http://www.wikidata.org/prop/direct/P135'), ('rname', 'Jean-Baptiste Carpeaux'), ('oname', 'movement')]
[('Num', '1'), ('obj', 'http://www.wikidata.org/prop/direct/P135'), ('rname', 'Étienne-Jules Ramey'), ('oname', 'movement')]
[('Num', '1'), ('obj', 'http://www.wikidata.org/prop/direct/P135'), ('rname', 'Constantin Daniel Rosenthal'), ('oname', 'movement')]
[('Num', '1'), ('obj', 'http://www.wikidata.org/prop/direct/P135'), ('rname', 'Edwin Landseer'), ('oname', 'movement')]
[('Num', '1'), ('obj', 'http://www.wikidata.org/prop/direct/P135'), ('rname', 'John-Étienne Chaponnière'), ('oname', 'movement')]
[('Num', '1'), ('obj', 'http://www.wikidata.org/prop/direct/P135'), ('rname', 'Auguste Clésinger'), ('oname', 'movement')]
[('Num', '1'), ('obj', 'http://www.wikidata.org/prop/direct/P135'), ('rname', 'Eugène Devéria'), ('oname', 'movement')]
[('Num', '1'), ('obj', 'http://www.wikidata.org/prop/direct/P135'), ('rname', 'Joseph Félon'), ('oname', 'movemen

In [153]:
# write your queries
#[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
queryString = """
SELECT DISTINCT ?scul ?sname
WHERE {
    ?scul ?obj wd:Q860861.
    ?scul sc:name ?sname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('scul', 'http://www.wikidata.org/entity/Q2520013'), ('sname', 'sculpture in the round')]
[('scul', 'http://www.wikidata.org/entity/Q8204840'), ('sname', 'Arqueta de Sant Cugat')]
[('scul', 'http://www.wikidata.org/entity/Q13723412'), ('sname', 'statue of Sacred Heart of Jesus Christ')]
[('scul', 'http://www.wikidata.org/entity/Q13723458'), ('sname', 'statue of Sacred Heart of Jesus Christ')]
[('scul', 'http://www.wikidata.org/entity/Q13723472'), ('sname', 'statue of Sacred Heart of Jesus Christ')]
[('scul', 'http://www.wikidata.org/entity/Q7982492'), ('sname', 'Wendell Willkie')]
[('scul', 'http://www.wikidata.org/entity/Q15725481'), ('sname', 'Amar Ekushey')]
[('scul', 'http://www.wikidata.org/entity/Q15726558'), ('sname', 'Machine with Concrete')]
[('scul', 'http://www.wikidata.org/entity/Q15726575'), ('sname', 'equestrian statue of Maréchal Foch')]
[('scul', 'http://www.wikidata.org/entity/Q15732310'), ('sname', 'statue of Sacred Heart of Jesus Christ')]
[('scul', 'http://

Final query for this task

There is no any data of sculptures where it made .

In [ ]:
# write your final query
# write your queries
#[('p', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
queryString = """
SELECT DISTINCT ?scul ?sname
WHERE {
    ?scul ?obj wd:Q860861.
    ?scul sc:name ?sname.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

## Task 6
For each European country find how many poets belong to Romanticism (the result set must be a list of triples country IRI, label and #poets).

In [221]:
#Using Europe BGP from other task 
#[('p', 'http://www.wikidata.org/entity/Q46'), ('name', 'Europe')]
queryString = """
SELECT DISTINCT ?cname ?m 
WHERE {
   # bind something
     wd:Q46 wdt:P527 ?p.
     ?p wdt:P527 ?cname.
   # get the label
   #?pn sc:name ?pname.
   ?cname sc:name ?m.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('cname', 'http://www.wikidata.org/entity/Q1410'), ('m', 'Gibraltar')]
[('cname', 'http://www.wikidata.org/entity/Q142'), ('m', 'France')]
[('cname', 'http://www.wikidata.org/entity/Q145'), ('m', 'United Kingdom')]
[('cname', 'http://www.wikidata.org/entity/Q183'), ('m', 'Germany')]
[('cname', 'http://www.wikidata.org/entity/Q184'), ('m', 'Belarus')]
[('cname', 'http://www.wikidata.org/entity/Q189'), ('m', 'Iceland')]
[('cname', 'http://www.wikidata.org/entity/Q191'), ('m', 'Estonia')]
[('cname', 'http://www.wikidata.org/entity/Q20'), ('m', 'Norway')]
[('cname', 'http://www.wikidata.org/entity/Q211'), ('m', 'Latvia')]
[('cname', 'http://www.wikidata.org/entity/Q212'), ('m', 'Ukraine')]
[('cname', 'http://www.wikidata.org/entity/Q213'), ('m', 'Czech Republic')]
[('cname', 'http://www.wikidata.org/entity/Q214'), ('m', 'Slovakia')]
[('cname', 'http://www.wikidata.org/entity/Q215'), ('m', 'Slovenia')]
[('cname', 'http://www.wikidata.org/entity/Q217'), ('m', 'Moldova')]
[('cname', 

In [229]:
#BGP of poets for exploring

queryString = """
SELECT DISTINCT ?poet ?poname
WHERE {
   # bind something
     ?poet ?o wd:Q49757.
    ?poet sc:name ?poname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q1373516'), ('poname', 'Eugénio de Castro')]
[('poet', 'http://www.wikidata.org/entity/Q589406'), ('poname', 'Empress Xiaoquancheng')]
[('poet', 'http://www.wikidata.org/entity/Q610608'), ('poname', 'António Aleixo')]
[('poet', 'http://www.wikidata.org/entity/Q710504'), ('poname', 'Bohuslav Balbín')]
[('poet', 'http://www.wikidata.org/entity/Q8188119'), ('poname', 'Adelina Lopes Vieira')]
[('poet', 'http://www.wikidata.org/entity/Q8201526'), ('poname', 'Antonio Márquez Salas')]
[('poet', 'http://www.wikidata.org/entity/Q8201629'), ('poname', 'Antonio Rendic')]
[('poet', 'http://www.wikidata.org/entity/Q8202094'), ('poname', 'Antônio de Castro Assumpção')]
[('poet', 'http://www.wikidata.org/entity/Q8246828'), ('poname', 'Bernat Català de Valleriola')]
[('poet', 'http://www.wikidata.org/entity/Q1191225'), ('poname', 'Tan Sitong')]
[('poet', 'http://www.wikidata.org/entity/Q4790571'), ('poname', 'Arifin C. Noer')]
[('poet', 'http://www.wik

In [231]:
#Exploring single poet

queryString = """
SELECT DISTINCT ?poet ?poname
WHERE {
   # bind something
    wd:Q231708 ?poet ?o.
    ?poet sc:name ?poname.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/prop/direct/P1005'), ('poname', 'Portuguese National Library ID')]
[('poet', 'http://www.wikidata.org/prop/direct/P1006'), ('poname', 'Nationale Thesaurus voor Auteurs ID')]
[('poet', 'http://www.wikidata.org/prop/direct/P1015'), ('poname', 'NORAF ID')]
[('poet', 'http://www.wikidata.org/prop/direct/P1038'), ('poname', 'relative')]
[('poet', 'http://www.wikidata.org/prop/direct/P106'), ('poname', 'occupation')]
[('poet', 'http://www.wikidata.org/prop/direct/P119'), ('poname', 'place of burial')]
[('poet', 'http://www.wikidata.org/prop/direct/P1196'), ('poname', 'manner of death')]
[('poet', 'http://www.wikidata.org/prop/direct/P1207'), ('poname', 'NUKAT ID')]
[('poet', 'http://www.wikidata.org/prop/direct/P1233'), ('poname', 'Internet Speculative Fiction Database author ID')]
[('poet', 'http://www.wikidata.org/prop/direct/P1263'), ('poname', 'NNDB people ID')]
[('poet', 'http://www.wikidata.org/prop/direct/P1273'), ('poname', 'CANTIC ID')]
[('

In [265]:
#Using country of citizenship P27 for getting country of poets working for Romanticism
#Q37068 - Romanticism
#Q49757 - Poet
queryString = """
SELECT DISTINCT ?poet ?poname ?coc ?cname
WHERE {
   # bind something
   ?poet ?obj wd:Q37068;
   wdt:P106 wd:Q49757;
    wdt:P27 ?coc.
    ?poet sc:name ?poname.
    ?coc sc:name ?cname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('poet', 'http://www.wikidata.org/entity/Q5932129'), ('poname', 'Joaquín José Cervino'), ('coc', 'http://www.wikidata.org/entity/Q29'), ('cname', 'Spain')]
[('poet', 'http://www.wikidata.org/entity/Q5258787'), ('poname', 'Tobias Barreto'), ('coc', 'http://www.wikidata.org/entity/Q155'), ('cname', 'Brazil')]
[('poet', 'http://www.wikidata.org/entity/Q5340521'), ('poname', 'Eduardo Asquerino'), ('coc', 'http://www.wikidata.org/entity/Q29'), ('cname', 'Spain')]
[('poet', 'http://www.wikidata.org/entity/Q573223'), ('poname', 'Patricio de la Escosura'), ('coc', 'http://www.wikidata.org/entity/Q29'), ('cname', 'Spain')]
[('poet', 'http://www.wikidata.org/entity/Q688529'), ('poname', 'Johannes Carsten Hauch'), ('coc', 'http://www.wikidata.org/entity/Q35'), ('cname', 'Denmark')]
[('poet', 'http://www.wikidata.org/entity/Q597298'), ('poname', 'Antoni Edward Odyniec'), ('coc', 'http://www.wikidata.org/entity/Q34266'), ('cname', 'Russian Empire')]
[('poet', 'http://www.wikidata.org/entit

In [266]:
#Restricting Countries to Europe only
queryString = """
SELECT DISTINCT (COUNT(?poet) AS ?NumOfPoets) ?coc ?cname
WHERE {
   # bind something
   ?poet ?obj wd:Q37068;
   wdt:P106 wd:Q49757;
    wdt:P27 ?coc.
    ?coc wdt:P30 wd:Q46.
    #?poet sc:name ?poname.
    ?coc sc:name ?cname.
}
GROUP BY ?coc ?cname
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfPoets', '1'), ('coc', 'http://www.wikidata.org/entity/Q156199'), ('cname', 'Electorate of Saxony')]
[('NumOfPoets', '4'), ('coc', 'http://www.wikidata.org/entity/Q218'), ('cname', 'Romania')]
[('NumOfPoets', '1'), ('coc', 'http://www.wikidata.org/entity/Q222'), ('cname', 'Albania')]
[('NumOfPoets', '1'), ('coc', 'http://www.wikidata.org/entity/Q28'), ('cname', 'Hungary')]
[('NumOfPoets', '1'), ('coc', 'http://www.wikidata.org/entity/Q958291'), ('cname', 'United Principalities')]
[('NumOfPoets', '1'), ('coc', 'http://www.wikidata.org/entity/Q12560'), ('cname', 'Ottoman Empire')]
[('NumOfPoets', '9'), ('coc', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany')]
[('NumOfPoets', '7'), ('coc', 'http://www.wikidata.org/entity/Q174193'), ('cname', 'United Kingdom of Great Britain and Ireland')]
[('NumOfPoets', '3'), ('coc', 'http://www.wikidata.org/entity/Q31'), ('cname', 'Belgium')]
[('NumOfPoets', '1'), ('coc', 'http://www.wikidata.org/entity/Q403'), ('cname', 'Serb

Final query for this task

In [267]:
# write your final query
queryString = """
SELECT DISTINCT (COUNT(?poet) AS ?NumOfPoets) ?coc ?cname
WHERE {
   # bind something
   ?poet ?obj wd:Q37068;
   wdt:P106 wd:Q49757;
    wdt:P27 ?coc.
    ?coc wdt:P30 wd:Q46.
    #?poet sc:name ?poname.
    ?coc sc:name ?cname.
}
GROUP BY ?coc ?cname
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('NumOfPoets', '1'), ('coc', 'http://www.wikidata.org/entity/Q156199'), ('cname', 'Electorate of Saxony')]
[('NumOfPoets', '4'), ('coc', 'http://www.wikidata.org/entity/Q218'), ('cname', 'Romania')]
[('NumOfPoets', '1'), ('coc', 'http://www.wikidata.org/entity/Q222'), ('cname', 'Albania')]
[('NumOfPoets', '1'), ('coc', 'http://www.wikidata.org/entity/Q28'), ('cname', 'Hungary')]
[('NumOfPoets', '1'), ('coc', 'http://www.wikidata.org/entity/Q958291'), ('cname', 'United Principalities')]
[('NumOfPoets', '1'), ('coc', 'http://www.wikidata.org/entity/Q12560'), ('cname', 'Ottoman Empire')]
[('NumOfPoets', '9'), ('coc', 'http://www.wikidata.org/entity/Q183'), ('cname', 'Germany')]
[('NumOfPoets', '7'), ('coc', 'http://www.wikidata.org/entity/Q174193'), ('cname', 'United Kingdom of Great Britain and Ireland')]
[('NumOfPoets', '3'), ('coc', 'http://www.wikidata.org/entity/Q31'), ('cname', 'Belgium')]
[('NumOfPoets', '1'), ('coc', 'http://www.wikidata.org/entity/Q403'), ('cname', 'Serb

## Task 7
Find the number of sculptures in the Louvre Museum belonging to Renaissance, Baroque, or Romanticism (return a number for each movement) (the result set must be a list of triples movement IRI, label and #sculptures).

In [293]:
# write your queries
# Exploring Louvre Museum BGP
queryString = """
SELECT DISTINCT ?oname ?name ?p ?obj
WHERE {
   # bind something
   wd:Q19675 ?obj ?p.
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname.
   FILTER(REGEX(?name,"Sculptures"^^xsd:string))
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('oname', 'has part'), ('name', 'Department of Sculptures of the Louvre'), ('p', 'http://www.wikidata.org/entity/Q3044772'), ('obj', 'http://www.wikidata.org/prop/direct/P527')]
1


In [216]:
# write your queries
# Exploring Department of Sculptures of the Louvre
queryString = """
SELECT DISTINCT ?p ?name ?obj ?oname
WHERE {
   # bind something
    ?obj ?p wd:Q3044772.
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic"), ('obj', 'http://www.wikidata.org/entity/Q8723838'), ('oname', 'Category:Sculptures of the Louvre')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('name', 'collection'), ('obj', 'http://www.wikidata.org/entity/Q17451496'), ('oname', 'Nymph with a Scorpion')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('name', 'collection'), ('obj', 'http://www.wikidata.org/entity/Q106350990'), ('oname', 'Farnese Flora')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('name', 'collection'), ('obj', 'http://www.wikidata.org/entity/Q18147625'), ('oname', 'Farnese Flora')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('name', 'collection'), ('obj', 'http://www.wikidata.org/entity/Q18147637'), ('oname', 'Theseus Fighting the Minotaur')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('name', 'collection'), ('obj', 'http://www.wikidata.org/entity/Q18147662'), ('oname', 'Tiber')]
[('

In [220]:
#Getting Specific Collection for exploring
queryString = """
SELECT DISTINCT ?p ?name ?obj ?oname
WHERE {
   # bind something
    wd:Q19753091 ?obj ?p.
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q40446'), ('name', 'nude'), ('obj', 'http://www.wikidata.org/prop/direct/P136'), ('oname', 'genre')]
[('p', 'http://www.wikidata.org/entity/Q318741'), ('name', 'Jean-Antoine Houdon'), ('obj', 'http://www.wikidata.org/prop/direct/P170'), ('oname', 'creator')]
[('p', 'http://www.wikidata.org/entity/Q10791'), ('name', 'nudity'), ('obj', 'http://www.wikidata.org/prop/direct/P180'), ('oname', 'depicts')]
[('p', 'http://www.wikidata.org/entity/Q132543'), ('name', 'Diana'), ('obj', 'http://www.wikidata.org/prop/direct/P180'), ('oname', 'depicts')]
[('p', 'http://www.wikidata.org/entity/Q46311'), ('name', 'bow'), ('obj', 'http://www.wikidata.org/prop/direct/P180'), ('oname', 'depicts')]
[('p', 'http://www.wikidata.org/entity/Q45922'), ('name', 'arrow'), ('obj', 'http://www.wikidata.org/prop/direct/P180'), ('oname', 'depicts')]
[('p', 'http://www.wikidata.org/entity/Q327496'), ('name', 'hairstyle'), ('obj', 'http://www.wikidata.org/prop/direct/P180

In [276]:
# write your queries
# Exploring sculpture BGP
#[('p', 'http://www.wikidata.org/entity/Q860861'), ('name', 'sculpture')
queryString = """
SELECT DISTINCT ?oname ?name ?p ?obj
WHERE {
   # bind something
    wd:Q860861 ?obj ?p .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('oname', 'field of work'), ('name', 'art of sculpture'), ('p', 'http://www.wikidata.org/entity/Q11634'), ('obj', 'http://www.wikidata.org/prop/direct/P101')]
[('oname', 'described by source'), ('name', 'Metropolitan Museum of Art Tagging Vocabulary'), ('p', 'http://www.wikidata.org/entity/Q106727050'), ('obj', 'http://www.wikidata.org/prop/direct/P1343')]
[('oname', 'different from'), ('name', 'art of sculpture'), ('p', 'http://www.wikidata.org/entity/Q11634'), ('obj', 'http://www.wikidata.org/prop/direct/P1889')]
[('oname', 'properties for this type'), ('name', 'Vanderkrogt.net Statues ID'), ('p', 'http://www.wikidata.org/entity/P3711'), ('obj', 'http://www.wikidata.org/prop/direct/P1963')]
[('oname', 'fabrication method'), ('name', 'sculpture technique'), ('p', 'http://www.wikidata.org/entity/Q21711025'), ('obj', 'http://www.wikidata.org/prop/direct/P2079')]
[('oname', 'subclass of'), ('name', 'visual artwork'), ('p', 'http://www.wikidata.org/entity/Q4502142'), ('obj', 'htt

Final query for this task

In [275]:
#[('p', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
#[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
queryString = """
SELECT DISTINCT ?oname ?name ?p ?obj
WHERE {
   # bind something
    wd:Q37068 ?obj ?p .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname.
   #FILTER(REGEX(?name,"sculpture"^^xsd:string))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('oname', 'described by source'), ('name', 'The Nuttall Encyclopædia'), ('p', 'http://www.wikidata.org/entity/Q3181656'), ('obj', 'http://www.wikidata.org/prop/direct/P1343')]
[('oname', 'described by source'), ('name', "Riemann's Music Dictionary (1901–1904)"), ('p', 'http://www.wikidata.org/entity/Q27680201'), ('obj', 'http://www.wikidata.org/prop/direct/P1343')]
[('oname', 'follows'), ('name', 'neoclassicism'), ('p', 'http://www.wikidata.org/entity/Q14378'), ('obj', 'http://www.wikidata.org/prop/direct/P155')]
[('oname', 'follows'), ('name', 'Pre-romanticism'), ('p', 'http://www.wikidata.org/entity/Q2614483'), ('obj', 'http://www.wikidata.org/prop/direct/P155')]
[('oname', 'followed by'), ('name', 'Post-romanticism'), ('p', 'http://www.wikidata.org/entity/Q1125039'), ('obj', 'http://www.wikidata.org/prop/direct/P156')]
[('oname', 'instance of'), ('name', 'literary movement'), ('p', 'http://www.wikidata.org/entity/Q3326717'), ('obj', 'http://www.wikidata.org/prop/direct/P31'

In [279]:
#[('p', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
#[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
#Exploring art style
queryString = """
SELECT DISTINCT ?oname ?name ?p ?obj
WHERE {
   # bind something
    wd:Q968159 ?obj ?p .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname.
   #FILTER(REGEX(?name,"sculpture"^^xsd:string))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('oname', "topic's main template"), ('name', 'Template:Infobox art movement'), ('p', 'http://www.wikidata.org/entity/Q14412130'), ('obj', 'http://www.wikidata.org/prop/direct/P1424')]
[('oname', 'different from'), ('name', 'art group'), ('p', 'http://www.wikidata.org/entity/Q4502119'), ('obj', 'http://www.wikidata.org/prop/direct/P1889')]
[('oname', 'has list'), ('name', 'list of art movements'), ('p', 'http://www.wikidata.org/entity/Q1293970'), ('obj', 'http://www.wikidata.org/prop/direct/P2354')]
[('oname', 'subclass of'), ('name', 'art'), ('p', 'http://www.wikidata.org/entity/Q735'), ('obj', 'http://www.wikidata.org/prop/direct/P279')]
[('oname', 'subclass of'), ('name', 'cultural movement'), ('p', 'http://www.wikidata.org/entity/Q2198855'), ('obj', 'http://www.wikidata.org/prop/direct/P279')]
[('oname', "topic's main category"), ('name', 'Category:Art movements'), ('p', 'http://www.wikidata.org/entity/Q7458526'), ('obj', 'http://www.wikidata.org/prop/direct/P910')]
6


In [280]:
#[('p', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
#[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
#Exploring art style
#Exploring art group
queryString = """
SELECT DISTINCT ?oname ?name ?p ?obj
WHERE {
   # bind something
    wd:Q4502119 ?obj ?p .
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname.
   #FILTER(REGEX(?name,"sculpture"^^xsd:string))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('oname', 'field of work'), ('name', 'art'), ('p', 'http://www.wikidata.org/entity/Q735'), ('obj', 'http://www.wikidata.org/prop/direct/P101')]
[('oname', 'different from'), ('name', 'art society'), ('p', 'http://www.wikidata.org/entity/Q12160242'), ('obj', 'http://www.wikidata.org/prop/direct/P1889')]
[('oname', 'different from'), ('name', 'artist collective'), ('p', 'http://www.wikidata.org/entity/Q1400264'), ('obj', 'http://www.wikidata.org/prop/direct/P1889')]
[('oname', 'different from'), ('name', 'art genre'), ('p', 'http://www.wikidata.org/entity/Q1792379'), ('obj', 'http://www.wikidata.org/prop/direct/P1889')]
[('oname', 'different from'), ('name', 'art movement'), ('p', 'http://www.wikidata.org/entity/Q968159'), ('obj', 'http://www.wikidata.org/prop/direct/P1889')]
[('oname', 'subclass of'), ('name', 'group of humans'), ('p', 'http://www.wikidata.org/entity/Q16334295'), ('obj', 'http://www.wikidata.org/prop/direct/P279')]
[('oname', 'subclass of'), ('name', 'voluntary

In [284]:
#[('p', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
#[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
#Exploring art style - art group - art genre
queryString = """
SELECT DISTINCT ?oname ?name ?p ?obj
WHERE {
   # bind something
     ?obj ?p wd:Q1792379.
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname.
   FILTER(REGEX(?oname,"visual"^^xsd:string))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('oname', 'visual art'), ('name', 'instance of'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('obj', 'http://www.wikidata.org/entity/Q36649')]
1


In [285]:
#[('p', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
#[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
#Exploring art style - art group - art genre - visual art
queryString = """
SELECT DISTINCT ?oname ?name ?p ?obj
WHERE {
   # bind something
     ?obj ?p wd:Q36649.
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname.
   FILTER(REGEX(?oname,"visual"^^xsd:string))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('oname', 'Pori city board of visual arts'), ('name', 'field of work'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('obj', 'http://www.wikidata.org/entity/Q21607642')]
[('oname', 'visual artist'), ('name', 'product or material produced'), ('p', 'http://www.wikidata.org/prop/direct/P1056'), ('obj', 'http://www.wikidata.org/entity/Q3391743')]
[('oname', 'visual art of the United States'), ('name', 'facet of'), ('p', 'http://www.wikidata.org/prop/direct/P1269'), ('obj', 'http://www.wikidata.org/entity/Q2311158')]
[('oname', 'Armenian visual arts'), ('name', 'facet of'), ('p', 'http://www.wikidata.org/prop/direct/P1269'), ('obj', 'http://www.wikidata.org/entity/Q4198730')]
[('oname', 'visual artwork'), ('name', 'different from'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('obj', 'http://www.wikidata.org/entity/Q4502142')]
[('oname', 'visual music'), ('name', 'subclass of'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('obj', 'http://www.wikidata.org/entity

In [291]:
#[('p', 'http://www.wikidata.org/entity/Q37068'), ('name', 'Romanticism')]
#[('p', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
#[('p', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
#Exploring art style - art group - art genre - visual art - visual artwork
queryString = """
SELECT DISTINCT ?oname ?name ?p ?obj
WHERE {
   # bind something
      ?obj ?p wd:Q4502142.
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?oname.
   FILTER(REGEX(?oname,"sculpture"^^xsd:string))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('oname', 'sculpture'), ('name', 'subclass of'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('obj', 'http://www.wikidata.org/entity/Q860861')]
[('oname', 'Model and Surrealist Sculpture (Modele et sculpture surrealiste)'), ('name', 'instance of'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('obj', 'http://www.wikidata.org/entity/Q91941050')]
[('oname', 'Political Prisoner (maquette for sculpture)'), ('name', 'instance of'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('obj', 'http://www.wikidata.org/entity/Q91931714')]
3


In [ ]:
# write your final query